# Librerías

In [1]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd
import string
import stanza
import nltk
import fs
import ast
import re
import unicodedata

In [2]:
# Download files
stanza.download("es") 
nltk.download('stopwords')
nltk.download('wordnet')

2024-11-28 07:53:35 INFO: Downloaded file to /root/stanza_resources/resources.json
2024-11-28 07:53:35 INFO: Downloading default packages for language: es (Spanish) ...


2024-11-28 07:53:40 INFO: Downloaded file to /root/stanza_resources/es/default.zip
2024-11-28 07:53:45 INFO: Finished downloading models and saved to /root/stanza_resources
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Lecturas

In [24]:
INTERIM_FILES_DIR = fs.open_fs("MCD-NLP-HPT/data/interim")
INTERIM_COMMENTS_ITEMS_DIR = INTERIM_FILES_DIR.getsyspath("Items.csv")

In [25]:
INTERIM_COMMENTS_CLEANED_DIR = INTERIM_FILES_DIR.getsyspath("Comments_cleaned.csv")
INTERIM_COMMENTS_CLEANED_DIR_WITHOUT_LEMMA_STEM = INTERIM_FILES_DIR.getsyspath("Comments_cleaned_only_sw.csv")

In [26]:
df_items = pd.read_csv(INTERIM_COMMENTS_ITEMS_DIR)
df_items

,kind,etag,id,snippet.channelId,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.channelId,snippet.topLevelComment.snippet.videoId,...,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,replies.comments,category_id,category_description
0,youtube#commentThread,35SzVbfAjbh-WxxI6CDcz8jJalE,Ugz0QmXyVLSM0nUK7kB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,yopv5G2xkuk2ZN4mdQMDcEtKmS4,Ugz0QmXyVLSM0nUK7kB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-07-15T12:41:43Z,2024-07-15T12:41:43Z,True,0,True,NaN,1,Quejas o sugerencias de mejora
1,youtube#commentThread,D5l-LBB9fD-VlBImz-EN9K8mTmM,UgyJhRBhVcbyStZdxjR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,avm1UM6tZkxbnMkp59t7_36GZrk,UgyJhRBhVcbyStZdxjR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-26T21:41:04Z,2024-05-26T21:41:04Z,True,0,True,NaN,7,Comentarios generales
2,youtube#commentThread,iKN6FQSPLo3xUlizd--6R0h2SN8,UgzT1q2am7kMEM5ckDp4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,g6vw71KL8rJyQPLYPCUCm4O6rbA,UgzT1q2am7kMEM5ckDp4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-23T22:06:21Z,2024-05-23T22:06:21Z,True,0,True,NaN,7,Comentarios generales
3,youtube#commentThread,hkErqy7RZOwAKS6YyNXqwPFTCAg,Ugyckucg38lqZFdZhaR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,soQFV-F5m7bRA03fMjI1IoKQrGs,Ugyckucg38lqZFdZhaR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-05T09:24:57Z,2024-05-05T09:24:57Z,True,0,True,NaN,6,Comentarios humorísticos o memes
4,youtube#commentThread,9c50dNDxQ96ZYyFK1aeJYj4ofnQ,UgzhZHVt-tEbDpbCvtB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,wLebngIiKaOrIw1ibNjosSTEyFc,UgzhZHVt-tEbDpbCvtB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-04-21T21:09:08Z,2024-04-21T21:09:08Z,True,0,True,NaN,5,Felicitaciones y agradecimientos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,youtube#commentThread,8wSxk2QdKgz2v0AUy3EsWvUOxSU,Ugyuq-tSoKFj7HEnAQ54AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,G_pL0x_Nmvi7C0kj8fHi_IWkcCA,Ugyuq-tSoKFj7HEnAQ54AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:46:46Z,2024-08-19T19:46:46Z,True,0,True,NaN,6,Comentarios humorísticos o memes
1022,youtube#commentThread,X4xqkZX0x2OmHUMWh4fe2ck4iGA,Ugy2h_vD89gf_8eeFph4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,fWBX1bGUjpM7nRaQcNKN1P7BpFQ,Ugy2h_vD89gf_8eeFph4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:33:41Z,2024-08-19T19:33:41Z,True,0,True,NaN,2,Propuestas y recomendaciones
1023,youtube#commentThread,btoZQh1KTx4_A0e8kLI3d4BAUmg,Ugx7vBivUc-z-n8H76t4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,CcywvY3FEol-BcpzJpjundijK6k,Ugx7vBivUc-z-n8H76t4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,1,2024-08-19T19:33:18Z,2024-08-19T19:33:18Z,True,0,True,NaN,1,Quejas o sugerencias de mejora
1024,youtube#commentThread,Xf-HPCYvwS0mb7JyqR4t_eoHZx4,Ugzvu7xFsIUo8ibNXPJ4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,FLjbXmI4uIIbM1ZQNW-hMSKWARM,Ugzvu7xFsIUo8ibNXPJ4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:13:00Z,2024-08-19T19:13:00Z,True,0,True,NaN,4,Correcciones o datos adicionales


In [27]:
df_cleaned = df_items[[
    'id', 
    'snippet.videoId',
    'snippet.topLevelComment.snippet.textOriginal',
    'snippet.topLevelComment.snippet.authorDisplayName',
    'snippet.topLevelComment.snippet.likeCount',
    'snippet.topLevelComment.snippet.publishedAt',
    'category_id',
    'category_description'
]].copy()
df_cleaned.columns = [
    'id', 
    'videoId',
    'textOriginal',
    'authorDisplayName',
    'likeCount',
    'publishedAt',
    'category_id',
    'category_description'
]
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos
...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales


# Limpieza

### Conversión a minusculas

In [28]:
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].str.lower()

### Componer teca

In [29]:
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].str.replace(r'\bteka\b', 'teca', 
regex=True, flags=re.IGNORECASE)

### Eliminar acentos

In [30]:
def quitar_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto)
        texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    return texto  

In [31]:
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].apply(quitar_acentos)

### Formato uniforme

In [32]:
def dar_formato_uniforme(texto):
    if isinstance(texto, str):  
        texto = unicodedata.normalize('NFKD', texto)
        texto = ''.join([c for c in texto if not unicodedata.combining(c)])
        texto = re.sub(r'[^\x00-\x7F]+', '', texto)  
        texto = texto.lower()
        texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

In [33]:
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].apply(dar_formato_uniforme)

### Eliminar comas

In [34]:
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].str.replace(r'[^\w\s]', '', regex=True) 
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].str.replace(r'\d+', '', regex=True)    

### Uniformar palabras de capitulo

In [35]:
palabras_a_reemplazar = r"\b(podcast|episodio|capitulo|video|cap)\b"
df_cleaned['textOriginal'] = df_cleaned['textOriginal'].apply(lambda x: re.sub(palabras_a_reemplazar, "episodio", x))


### Revisión de frecuencias

In [36]:
from collections import Counter
text_combined = " ".join(df_cleaned['textOriginal'].dropna())
word_list = text_combined.split()
word_frequency = Counter(word_list)
word_freq_df = pd.DataFrame(word_frequency.items(), columns=['Palabra', 'Frecuencia']).sort_values(by='Frecuencia', ascending=False)

#### Creando stopwords

In [37]:
freq_less_3 = word_freq_df[word_freq_df['Frecuencia']<3]
freq_less_3_sw = freq_less_3['Palabra'].tolist()
sw_to_add =['', '3', 'a', 'aca', 'acabo', 'ademas', 'ahi', 
'ahora', 'alguien', 'algun', 'algunos', 'alio', 'antes', 
'apenas', 'aqui', 'asi', 'asuncion', 'aun', 'aunque', 
'ayudarlo', 'bajo', 'bastante', 'be', 'blanco', 'boliviaaaaaaa', 
'cada', 'cambien', 'capitulos', 'casi', 'chicooooooooos', 
'cierto', 'como', 'contra', 'cosas', 'creo', 'd', 'da', 'de', 'deberian', 
'decir', 'decirlo', 'dejar', 'dejaron', 'despues', 'dice', 'dicen', 
'dijo', 'donde', 'dos', 'durante', 'ellos', 'empece', 'entonces',
'eres', 'es', 'escucha', 'escuchar', 'escucharlos', 'escucho', 'esperando', 
'espero', 'estar', 'estuvo', 'falta', 'favor', 'forma', 'fueron', 
'gracias', 'gustaria', 'gusto', 'ha', 'haber', 'habla', 'hablando', 'hablaran', 
'hablen', 'hace', 'hacen', 'hacer', 'hago', 'he', 'hecho', 'hicieron',
'hizo', 'hola', 'hoy', 'id10ta', 'incluso', 'inicio', 'inviten', 
'ir', 'jaja', 'jajaja', 'jajajaja', 'justo', 'k', 'la', 
'llama', 'llegar', 'llevo', 'luego', 'madre', 'manera', 'mapita', 
'mas', 'me', 'mi', 'mientras', 'mil', 'mis', 'mismo', 'mucha', 'muchas', 
'mucho', 'muchos', 'muy', 'neta', 'ningun', 'nombre', 'nombres', 'nos', 
'nuestra', 'nunca', 'of', 'otro', 'otros', 'p', 'pa', 'palabra', 'palabras', 
'parece', 'pasado', 'pasar', 'paso', 'pd', 'pense', 'plis', 'poco', 'poder', 
'poner', 'porfa', 'primer', 'primera', 'primero', 'primeros', 'pt4', 'puedan', 'puede', 
'pueden', 'puedo', 'pues', 'q', 'queria', 'querian', 'quiero', 'ra', 'recomiendo', 
'sabe', 'saben', 'saber', 'sacan', 'san', 'se', 'sea', 'seguir', 'segun', 
'segunda', 'semana', 'ser', 'seria', 'si', 'siempre', 'sigan', 'sigo', 'sigue', 
'sino', 'solo', 'solterooooooooooooooo', 'son','tal', 'tan', 'te', 'tenemos', 
'tener', 'tengo', 'tenia', 'tenian', 'the', 'tienen', 'tipo', 'titere', 'toda', 
'tontos', 'trabajo', 'tu','ustedes', 'va', 'vale', 'van', 'veces', 'vengo', 
'veo', 'ver', 'verlos', 'vez', 'vidi', 'viendo', 'vine', 'vini2', 'voy', 'wey', 
'xd', 'y', 'weeeeey', 'debia', 'explica', 'pasaron', 'buscando', 'alla', 'largo', 'd',
'guapos', 'termino', 'nino', 'entender', 'varios', 'senor', 'otra', 'anastasio', 'partes', 'dejen',
'debe', 'revamp', 'puse','comentarios', 'lean','vean', 'reir', 'tuvieron', 'pudo', 'cae', 'hermanos',
'diciendo', 'repetir', 'porfavor', 'ah', 'quede', 'ultimo', 'rato', 'idea', 'invitados', 'salir',
'gustan', 'podria', 'descubri', 'dar', 'terminar', 'chingo', 'lado', 'milhouse',  'chismecito', 
'cuentan', 'ultimos', 'dieron', 'iba', 'entendido', 'estuve', 'jejeje', 'conocia', 'norte',
'quedo', 'contar', 'nuestros', 'llego', 'pronunciar', 'procede', 'par', 'do', 'aprendo',
'dormir', 'partir', 'mayor', 'age', 'jajajajaja', 'claro', 'numero', 'debio', 'ven', 'pudieron',
'alguno', 'embargo', 'sera', 'patreon', 'alguna', 'vaya', 'encontrar', 'defenza', 'debajo', 'totos',
'correccion', 'oir', 'oye', 'interrumpir', 'todavia', 'usaban', 'ja', 'tambien', 'dia', 'pt', 'gusta', 'ganas', 'hagan', 'vi', 'estan', 
'episodios', 'invitados', 'saludo', 'hablar', 'habla', 'estaria', 'tk', 'alv', 'habia']
stopwords_spanish = stopwords.words('spanish')
lista_sw = []
lista_sw.extend(stopwords_spanish)
lista_sw.extend(sw_to_add)
lista_sw.extend(freq_less_3_sw)

In [46]:
word_freq_df

,Palabra,Frecuencia
18,de,1057
9,que,719
52,la,634
60,el,595
51,y,591
...,...,...
2622,peloponeso,1
809,nitro,1
808,metanle,1
2626,bailen,1


In [47]:
#word_freq_df.to_csv("word_freq.csv", encoding='utf-8', index=False)

In [38]:
texto = df_cleaned['textOriginal'][52]
texto

' hay un anime donde su protagonista se llama nike esta hermoso'

# Tokenizar y eliminar sw

In [39]:
df_cleaned['Tokens'] = df_cleaned['textOriginal'].apply(lambda x: x.split())

In [19]:
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,me gusta su episodio pero solo he escuchado lo...,@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[me, gusta, su, episodio, pero, solo, he, escu..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,les voy a enviar una foto de tarapaca,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[les, voy, a, enviar, una, foto, de, tarapaca]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,entonces teca esta solterooooooooooooooo woooooo,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[entonces, teca, esta, solterooooooooooooooo, ..."
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,me encantan sus episodio los escucho en spotif...,@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[me, encantan, sus, episodio, los, escucho, en..."
...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[justo, a, tiempo, para, hacerme, pndja, en, e..."
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,ya que son otakus declarados les recomiendo el...,@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[ya, que, son, otakus, declarados, les, recomi..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,pero dejen hablar al invitado chavos banda d,@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[pero, dejen, hablar, al, invitado, chavos, ba..."
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,el primer metal no seria el na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[el, primer, metal, no, seria, el, na, cuando,..."


Removing stopwords

In [40]:
# Remove stopwords and punctuation
[ word for word in df_cleaned['Tokens'][52] if word not in lista_sw]

['anime']

In [41]:
df_cleaned['Tokens_without_stopwords'] = df_cleaned['Tokens'].apply(lambda x: [word for word in x if word not in lista_sw])
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens,Tokens_without_stopwords
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,me gusta su episodio pero solo he escuchado lo...,@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[me, gusta, su, episodio, pero, solo, he, escu...","[episodio, escuchado, youtube, spotify, youtube]"
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey],[]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,les voy a enviar una foto de tarapaca,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[les, voy, a, enviar, una, foto, de, tarapaca]","[foto, tarapaca]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,entonces teca esta solterooooooooooooooo woooooo,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[entonces, teca, esta, solterooooooooooooooo, ...",[teca]
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,me encantan sus episodio los escucho en spotif...,@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[me, encantan, sus, episodio, los, escucho, en...","[encantan, episodio, spotify, viajo, conoci, o..."
...,...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[justo, a, tiempo, para, hacerme, pndja, en, e...","[tiempo, bien, invitado]"
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,ya que son otakus declarados les recomiendo el...,@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[ya, que, son, otakus, declarados, les, recomi...","[anime, anime, bien, episodio]"
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,pero dejen hablar al invitado chavos banda d,@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[pero, dejen, hablar, al, invitado, chavos, ba...","[invitado, chavos, banda]"
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,el primer metal no seria el na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[el, primer, metal, no, seria, el, na, cuando,...","[metal, sal]"


In [42]:
df_cleaned.to_csv(INTERIM_COMMENTS_CLEANED_DIR_WITHOUT_LEMMA_STEM, encoding='utf-8', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2820e488-6f1b-466d-af14-a66826f012e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>